In [1]:
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from collections import Counter
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

/home/zayd/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Dataset loading

news_test = datasets.load_files('./category', encoding='utf-8')
x_train, x_test, y_train, y_test = train_test_split(news_test.data, news_test.target, test_size=0.2)

print(len(news_test.data))

102099


In [3]:
def get_word_2_index(vocab):
    word2index = {}
    for i,word in enumerate(vocab):
        word2index[word] = i

    return word2index


def get_batch(df_train, df_test,i,batch_size):
    batches = []
    results = []
    texts = df_train[i*batch_size:i*batch_size+batch_size]
    categories = df_test[i*batch_size:i*batch_size+batch_size]
    
    for text in texts:
        layer = np.zeros(total_words,dtype=float)
        for word in text.split(' '):
            layer[word2index[word]] += 1

        batches.append(layer)

    for category in categories:
        y = np.zeros((12),dtype=float)

        for i in range(12):
            if category == i:
                y[i] = 1.
                
        results.append(y)

    return np.array(batches),np.array(results)

In [4]:
vocab = Counter()

for text in x_train:
    for word in text.split(' '):
        vocab[word]+=1

for text in x_test:
    for word in text.split(' '):
        vocab[word]+=1

total_words = len(vocab)

print(total_words)

word2index = get_word_2_index(vocab)


865455


In [ ]:
# Neural network model 

# Parameters
learning_rate = 0.01
epochs = 10
batch_size = 150
display_step = 1

input_num = total_words
output_num = 12

layer_1_nodes = 50
layer_2_nodes = 100
layer_3_nodes = 50

with tf.variable_scope('input'):
    X = tf.placeholder(tf.float32, shape=(None, input_num))
    
# Layer 1
with tf.variable_scope('layer_1'):
    weights = tf.get_variable("weights1", shape=[input_num, layer_1_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases1", shape=[layer_1_nodes], initializer=tf.zeros_initializer())
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)

# Layer 2
with tf.variable_scope('layer_2'):
    weights = tf.get_variable("weights2", shape=[layer_1_nodes, layer_2_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases2", shape=[layer_2_nodes], initializer=tf.zeros_initializer())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)

# Layer 3
with tf.variable_scope('layer_3'):
    weights = tf.get_variable("weights3", shape=[layer_2_nodes, layer_3_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases3", shape=[layer_3_nodes], initializer=tf.zeros_initializer())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)

# Output Layer
with tf.variable_scope('output'):
    weights = tf.get_variable("weights4", shape=[layer_3_nodes, output_num], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases4", shape=[output_num], initializer=tf.zeros_initializer())
    prediction = tf.matmul(layer_3_output, weights) + biases

    
# The cost function of the neural network

with tf.variable_scope('cost'):
    Y = tf.placeholder(tf.float32, shape=(None, output_num))
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=Y))

# The optimizer function

with tf.variable_scope('train'):
    
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [ ]:
# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # Training cycle
    
    for epoch in range(epochs):
        avg_cost = 0.
        total_batch = int(len(x_train)/batch_size)

        # Loop over all batches

        for i in range(total_batch):
            batch_x,batch_y = get_batch(x_train,y_train,i,batch_size)

            c,_ = sess.run([cost,optimizer], feed_dict={X: batch_x,Y:batch_y})
            avg_cost += c / total_batch
            

        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "loss=", "{:.9f}".format(avg_cost))

    

        '''# Test model
        correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))

        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

        total_test_data = len(x_test)
        batch_x_test,batch_y_test = get_batch(x_test,y_test,0,total_test_data)
        print("Accuracy:", accuracy.eval({input_tensor: batch_x_test, output_tensor: batch_y_test}))'''
        
    print("Optimization Finished!")

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0001 loss= 0.815284426
Epoch: 0002 loss= 0.388721734
Epoch: 0003 loss= 0.316627504
Epoch: 0004 loss= 0.287953078
Epoch: 0005 loss= 0.277781004
